# Numerically Solving the Davis-Skodje Equation

## Imports

In [8]:
# Mathematical tools
using LinearAlgebra
using DifferentialEquations
using GeometryBasics
using ForwardDiff


# General tools
using BenchmarkTools

# Plotting
using Colors, ColorSchemes
using CairoMakie
# Use CairoMakie with SVG output
CairoMakie.activate!(type = "svg")

    Updating registry at `C:\Users\leona\.julia\registries\General.toml`


   Resolving package versions...


    Updating `C:\Users\leona\.julia\environments\v1.9\Project.toml`
  [5c1252a2] + GeometryBasics v0.4.9
  No Changes to `C:\Users\leona\.julia\environments\v1.9\Manifest.toml`


## Davis-Skodje-Equation

In [2]:
function davis_skodje!(du, u, gamma, t)
    du[1] = -u[1]
    du[2] = -gamma * u[2] + ((gamma - 1) * u[1] + gamma * u[1]^2) / (1 + u[1]^2)
end

function wdavis_skodje!(du, u)
    gamma = 40
    du[1] = -u[1]
    du[2] = -gamma * u[2] + ((gamma - 1) * u[1] + gamma * u[1]^2) / (1 + u[1]^2)
end

wdavis_skodje! (generic function with 1 method)

## Plot functionality

In [3]:
function plot_skodje(ax, gamma)
    dsk(u) = Point2f(
        -u[1],
        -gamma * u[2] + ((gamma - 1) * u[1] + gamma * u[1]^2) / (1 + u[1]^2)
    )
    streamplot!(dsk, 0..4, 0..2, colormap = :magma, ax = ax)
end

plot_skodje (generic function with 1 method)

In [4]:
function euler(f, u0, dt, num_steps, gamma)
    """
    Explicit Euler method for solving an ODE.
    """
    # Initialize array to store solution
    u = zeros(length(u0), num_steps)
    u[:, 1] .= u0
    # Initialize derivative
    du = zeros(size(u0))
    for i in 2:num_steps
        # Compute derivative
        f(du, u[:, i - 1], gamma, 0)
        # Update solution
        u[:, i] .= u[:, i - 1] + dt * du
    end
    return u
end

function implicit_euler(f, u0, dt, num_steps, gamma)
    # Initialize array to store solution
    u = zeros(length(u0), num_steps)
    all_step_history = []
    u[:, 1] .= u0
    # Initialize jacobian
    for i in 2:num_steps
        u[:, i] = u[:, i - 1]
        for j in 1:20
            # Compute Jacobian
            du0 = zeros(size(u0))
            J = ForwardDiff.jacobian(f, du0, u[:, i])
            G = I - dt * J
            GF = factorize(G)
            f(du0, u[:, i])
            g_zero = u[:, i] - u[:, i - 1] - dt * du0
            u[:, i] = u[:, i] - GF \ g_zero
            # all step history, for plotting
            push!(all_step_history, u[:, i])
            if norm(g_zero) < 1e-3
                break
            end
        end
    end
    return u, reduce(hcat, all_step_history)
end

implicit_euler (generic function with 1 method)

In [5]:
u0 = [4, 0]
gamma = 40
tspan = (0.0, 5)
prob = ODEProblem(davis_skodje!, u0, tspan, gamma, saveat = 0.1)
sol = solve(prob)
rsol = reduce(hcat,sol.u)
# print(sol.destats)
# println(sol.alg) # uses stiff method

sol_euler = euler(davis_skodje!, u0, 0.049, 100, gamma)
sol_imp_euler, all_step_history = implicit_euler(wdavis_skodje!, u0, 0.4, 100, gamma)

f = Figure(fontsize = 20)
ax = Axis(f[1, 1], xlabel = L"u_1", ylabel = L"u_2", title = "Stream plot of the Davis-Skodje equation", xlabelsize = 40, ylabelsize = 40)

plot_skodje(ax, gamma)
lines!(sol_imp_euler[1, :], sol_imp_euler[2, :], color= :royalblue3, linewidth = 4, linestyle = :dash)
plot!(sol_imp_euler[1, :], sol_imp_euler[2, :], color= :royalblue3, markersize = 20, label = "Implicit Euler, dt = 0.4")
# lines!(sol_euler[1, :], sol_euler[2, :], color = :indianred, markersize = 20, linestyle = :dash, alpha = 0.1)
# plot!(sol_euler[1, :], sol_euler[2, :], color = :indianred, markersize = 20, label = "Explicit Euler, dt = 0.049", alpha = 0.1)
# plot all_step_history
plot!(all_step_history[1, :], all_step_history[2, :], color = :deeppink3, markersize = 10, linestyle = :dash, alpha = 0.1, label = "Newton iteration steps")
axislegend(ax, merge = true, unique = true)
xlims!(0, 4)
ylims!(0, 2)

save("davis_skodje2.svg", f)

UndefVarError: UndefVarError: `ForwardDiff` not defined

In [6]:
display(f)

UndefVarError: UndefVarError: `f` not defined